In [ ]:
import sys
sys.path.insert(0, '../')

In [ ]:
from neural_map import NeuralMap, _plot
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df = pd.concat([pd.read_csv("datasets/table_219.csv"), pd.read_csv("datasets/table_220.csv"), pd.read_csv("datasets/table_221.csv")]).drop(['Sex', 'Delta 15 N (o/oo)', 'Delta 13 C (o/oo)', 'Comments'], axis=1).dropna().reset_index(drop=True)
l_columns = ['studyName', 'Sample Number', 'Species', 'Region', 'Island', 'Stage', 'Individual ID', 'Clutch Completion', 'Date Egg']
col = 'Species'

In [ ]:
scaler = MinMaxScaler()
data = scaler.fit_transform(df.drop(l_columns, axis=1).values)

data.shape

In [ ]:
som = NeuralMap(z=data.shape[1], x=5, y=5, metric='correlation')

In [ ]:
som.train(data=data, verbose=True, n_epochs=30, radius_decay_function='exponential')
som.evaluate(data=data)

In [ ]:
som.plot_analysis(data, labels=df[col].values, min_cluster_size=3, display_empty_nodes=False)

In [ ]:
som.plot_unified_distance_matrix(detailed=True, borders=True)

In [ ]:
actfreq, q, mean_distance = som.analyse(data)
umatrix, distance_matrix = som.get_unified_distance_matrix()
labels, probabilities, outlier_socer = som.hdbscan(min_cluster_size=3)

_plot.tiles(som.positions, som.hexagonal, umatrix[..., -1], title='distancia')
_plot.tiles(som.positions, som.hexagonal, actfreq, title='frecuencia de actvación')
_plot.tiles(som.positions, som.hexagonal, q, title='error de cuantización')
_plot.tiles(som.positions, som.hexagonal, mean_distance, title='distancia media')
_plot.tiles(som.positions, som.hexagonal, probabilities, title = 'probabilidad de prtenencia')
_plot.tiles(som.positions, som.hexagonal, outlier_socer, title = 'outiler score')

In [ ]:
clusters = 3

labels, centers = som.k_means(clusters)
_plot.tiles(som.positions, som.hexagonal, labels, norm=False, title='Clustering con K-means. Valor de K = ' + str(clusters), labels=list(range(clusters)))

labels, centers = som.k_medoids(clusters)
_plot.tiles(som.positions, som.hexagonal, labels, norm=False, title='Clustering con K-medoids. Valor de K = ' + str(clusters), labels=list(range(clusters)))

In [ ]:
som.plot_cluster_weights_vectors(cluster=0)
som.plot_weights_vector(node_index=(0, 0))